In [1]:
import os
import sys

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()
sys.path.append(os.path.abspath("Efficient-Computing/Detection/Gold-YOLO"))

In [2]:
from utils import create_all_folders, Folders

create_all_folders()

In [3]:
from layers import AMF_GD_YOLOv8
import torch
from utils import Folders
from data_processing import ImageData
from training_parameters import (
    class_names,
    class_indices,
    class_colors,
    transform_pixel,
    transform_spatial,
)
from training import (
    TreeDataset,
    train_and_validate,
    create_and_save_splitted_datasets,
    load_tree_datasets_from_split,
)
import multiprocessing as mp

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = (
    "garbage_collection_threshold:0.6,max_split_size_mb:512"
)
print(os.environ.get("PYTORCH_CUDA_ALLOC_CONF"))

garbage_collection_threshold:0.6,max_split_size_mb:512


In [5]:
model = AMF_GD_YOLOv8(3, 1, device=device, scale="n", class_names=class_names).to(
    device
)

# Find and split the data

full_image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
resolution = 0.08

image_data = ImageData(full_image_path)

annotations_folder_path = os.path.join(
    Folders.CROPPED_ANNOTS.value, image_data.base_name
)
rgb_folder_path = os.path.join(Folders.CROPPED_IMAGES.value, image_data.base_name)
chm_folder_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "unfiltered",
    "cropped",
    image_data.coord_name,
)

sets_ratios = [3, 1]
sets_names = ["training", "validation"]
data_split_file_path = "../data/others/data_split.json"
dismissed_classes = ["Tree_unsure"]

create_and_save_splitted_datasets(
    rgb_folder_path,
    chm_folder_path,
    annotations_folder_path,
    sets_ratios,
    sets_names,
    data_split_file_path,
    random_seed=42,
)

datasets = load_tree_datasets_from_split(
    data_split_file_path,
    class_indices,
    class_colors,
    dismissed_classes=dismissed_classes,
    transform_spatial_training=transform_spatial,
    transform_pixel_training=transform_pixel,
)

# Training parameters

lr = 1e-2
epochs = 100

batch_size = 12
num_workers = mp.cpu_count()
accumulate = 12

final_model = train_and_validate(
    model=model,
    datasets=datasets,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
    num_workers=num_workers,
    accumulate=accumulate,
    device=device,
)
state_dict = final_model.state_dict()
index = 1
while os.path.exists(f"../models/amf_gd_yolov8/trained_model_{index}.pt"):
    index += 1
torch.save(state_dict, f"../models/amf_gd_yolov8/trained_model_{index}.pt")

Output()

/home/alexandre/miniforge3/envs/tree-segment/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1716579264291/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [6]:
from time import time
import multiprocessing as mp
from training import TreeDataLoader

batch_size = 8

for num_workers in range(2, mp.cpu_count() + 2, 2):
    train_loader = TreeDataLoader(
        datasets["training"],
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    start = time()
    for epoch in range(1, 3):
        for i, data in enumerate(train_loader, 0):
            pass
    end = time()
    print(f"Finish with: {end - start} second, num_workers={num_workers}")

Finish with: 9.793368101119995 second, num_workers=2
Finish with: 7.073188543319702 second, num_workers=4
Finish with: 6.6157708168029785 second, num_workers=6
Finish with: 6.237012624740601 second, num_workers=8


In [7]:
# from training import test_save_output_image, TreeDataLoader

# model = AMF_GD_YOLOv8(3, 1, device=device, scale="n", class_names=class_names).to(
#     device
# )

# state_dict = torch.load("../models/amf_gd_yolov8/trained_model_1.pt")
# model.load_state_dict(state_dict)

# test_loader = TreeDataLoader(
#     val_dataset,
#     batch_size=1,
#     shuffle=True,
#     num_workers=1,
#     pin_memory=True,
# )

# test_save_output_image(model, test_loader, -1, device, 10)